In [ ]:
import numpy as np
x_npy_path = "default of credit/test/X.npy" 
y_npy_path = "default of credit/test/y.npy"

y = np.load(y_npy_path)

X_full = np.load(x_npy_path); y = np.load(y_npy_path)
print(X_full.shape)
print(y.shape)
D = X_full.shape[1]

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
seed = np.random.randint(0, 2**32 - 1)
n_jobs = -1
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_full, y), start=1):
    X_tr, X_te = X_full[tr_idx], X_full[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy: {acc:.4f}")

print(f"Mean accuracy (2-fold): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")


(30000, 23)
(30000,)
Fold 1 accuracy: 0.8143
Fold 2 accuracy: 0.8236
Mean accuracy (2-fold): 0.8189 ± 0.0066


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from Flow_Disentangle_Feature_Importance import FlowMatchingModel

model = FlowMatchingModel(
    X=X_full,
    dim=D,
    device=device,
    hidden_dim=64,        
    time_embed_dim=64,     
    num_blocks=1,
    use_bn=False,
    seed=seed
)
model.fit(num_steps=50000, batch_size=256, lr=1e-3, show_plot=False)

from Flow_Disentangle_Feature_Importance import CPI_Flow_Model_Estimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator3 = CPI_Flow_Model_Estimator_cls(
    random_state=seed,
    regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                ),
    flow_model=model
)

phi_x_cpi, se_x_cpi = estimator3.importance(X_full, y)

print("Feature\tFDFI φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_x_cpi, se_x_cpi)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of FDFI: {D* np.mean(phi_x_cpi)}")

phi_x_cpi_test = phi_x_cpi 

se_x_cpi_test = se_x_cpi 

z_score_x_cpi = phi_x_cpi_test / se_x_cpi_test

p_value_x_cpi = 1 - norm.cdf(z_score_x_cpi)
rounded_p_value_x_cpi = np.round(p_value_x_cpi, 3)

print(rounded_p_value_x_cpi)

alpha = 0.05 / 23  

mask = (p_value_x_cpi <= alpha).astype(int)

print(mask)



/root/miniconda3/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
CPI@Z (resample) → decode → X (cls CE): 100%|██████████| 23/23 [01:25<00:00,  3.72s/it]
/root/autodl-tmp/Module_Flow_Disentangle_Feature_Importance/Flow_Disentangle_Feature_Importance/Flow_Matching/flow_matching.py:213: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  t_tensor = torch.tensor([[float(t_scalar)]], dtype=torch.float32, device=self.device)
CPI@Z (resample) → decode → X (cls CE): 100%|██████████| 23/23 [01:22<00:00,  3.58s/it]


Feature	FDFI φ	StdError
  0	 0.0064	 0.0006
  1	 0.0691	 0.0410
  2	 0.0013	 0.0003
  3	 0.0001	 0.0000
  4	 0.0012	 0.0003
  5	 0.0346	 0.0013
  6	 0.0169	 0.0008
  7	 0.0156	 0.0008
  8	 0.0156	 0.0009
  9	 0.0141	 0.0008
 10	 0.0125	 0.0008
 11	 0.0035	 0.0005
 12	 0.0034	 0.0005
 13	 0.0032	 0.0005
 14	 0.0030	 0.0005
 15	 0.0030	 0.0005
 16	 0.0029	 0.0005
 17	 0.0064	 0.0010
 18	 0.0046	 0.0008
 19	 0.0045	 0.0007
 20	 0.0043	 0.0007
 21	 0.0044	 0.0010
 22	 0.0042	 0.0009
Sum of FDFI: 0.23487964148015744
[0.    0.046 0.    0.023 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
[1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


# DFI

In [ ]:
from Flow_Disentangle_Feature_Importance import  DFIEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np
print(X_full.shape)
estimator4 = DFIEstimator_cls(
    random_state=seed,
    regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )
)

phi_x_dfi, se_x_dfi = estimator4.importance(X_full, y)


print("Feature\tDFI φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_x_dfi, se_x_dfi)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of DFI: {D* np.mean(phi_x_dfi)}")

phi_x_dfi_test = phi_x_dfi 

se_x_dfi_test = se_x_dfi 

z_score_x_dfi = phi_x_dfi_test / se_x_dfi_test


p_value_x_dfi = 1 - norm.cdf(z_score_x_dfi)
rounded_p_value_x_dfi = np.round(p_value_x_dfi, 3)

print(rounded_p_value_x_dfi)

alpha = 0.05 / 23 

mask = (p_value_x_dfi <= alpha).astype(int)

print(mask)



(30000, 23)
Feature	DFI φ	StdError
  0	 0.0186	 0.0017
  1	 0.0003	 0.0006
  2	 0.0011	 0.0009
  3	 0.0000	 0.0000
  4	 0.0009	 0.0009
  5	 0.0351	 0.0021
  6	 0.0159	 0.0016
  7	 0.0087	 0.0013
  8	 0.0080	 0.0012
  9	 0.0076	 0.0012
 10	 0.0065	 0.0012
 11	 0.0037	 0.0012
 12	 0.0029	 0.0011
 13	 0.0024	 0.0011
 14	 0.0026	 0.0011
 15	 0.0025	 0.0011
 16	 0.0023	 0.0011
 17	 0.0037	 0.0011
 18	 0.0029	 0.0010
 19	 0.0029	 0.0011
 20	 0.0023	 0.0010
 21	 0.0018	 0.0010
 22	 0.0021	 0.0010
Sum of DFI: 0.13499047386512786
[0.    0.307 0.106 0.5   0.143 0.    0.    0.    0.    0.    0.    0.001
 0.005 0.015 0.012 0.014 0.02  0.001 0.002 0.003 0.012 0.037 0.021]
[1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0]


# ad hoc

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

print("X shape:", X_full.shape)
print("y shape:", y.shape)

n, D = X_full.shape

ctg_names = [
    "LB","AC","FM","UC","DL","DS","DP",
    "ASTV","MSTV","ALTV","MLTV",
    "Width","Min","Max","Nmax","Nzeros","Mode","Mean","Median","Variance","Tendency"
]
feature_names = ctg_names if D == 21 else [f"f{i}" for i in range(D)]
X_df = pd.DataFrame(X_full, columns=feature_names)

corr = spearmanr(X_df).correlation
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1.0)
corr = np.nan_to_num(corr, nan=0.0, posinf=0.0, neginf=0.0)

distance_matrix = 1.0 - np.abs(corr)
linkage = hierarchy.ward(squareform(distance_matrix))

t = 0.1  # 1.075
cluster_ids = hierarchy.fcluster(linkage, t, criterion="distance")
cluster_to_ids = defaultdict(list)
for idx, cid in enumerate(cluster_ids):
    cluster_to_ids[cid].append(idx)

def select_representatives(
    X: np.ndarray,
    y: np.ndarray,
    cluster_to_ids: dict,
    corr_abs: np.ndarray,           # |spearman|
    dist: np.ndarray,               # 1 - |spearman|
    strategy: str = "medoid",
    random_state: int = 0
):
    rng = np.random.RandomState(random_state)
    selected = []

    for cid, ids in cluster_to_ids.items():
        ids = list(ids)
        if len(ids) == 1:
            selected.append(ids[0])
            continue

        if strategy == "first":
            chosen = ids[0]

        elif strategy == "medoid":
            subD = dist[np.ix_(ids, ids)]
            sums = np.sum(subD, axis=1)
            chosen = ids[int(np.argmin(sums))]
        selected.append(chosen)

    return selected

strategy = "medoid"   
selected_feature_idx = select_representatives(
    X_full, y, cluster_to_ids,
    corr_abs=np.abs(corr),
    dist=distance_matrix,
    strategy=strategy,
    random_state=seed
)
selected_feature_names = [feature_names[i] for i in selected_feature_idx]
print(f"[{strategy}] selected numbers: {len(selected_feature_idx)}")
print(" selected names:", selected_feature_names)

X_subset = X_full[:, selected_feature_idx]

seed = seed
n_jobs = -1
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")


X shape: (30000, 23)
y shape: (30000,)
[medoid] selected numbers: 20
 selected names: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f13', 'f15', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22']
Fold 1 accuracy (selected features): 0.8162
Fold 2 accuracy (selected features): 0.8232
Mean accuracy (2-fold, selected features): 0.8197 ± 0.0049


In [ ]:
from Flow_Disentangle_Feature_Importance import CPIEstimator_cls
from scipy.stats import norm
from sklearn.ensemble import RandomForestClassifier
import numpy as np

estimator5 = CPIEstimator_cls(
        random_state=seed,
        regressor =  RandomForestClassifier(
                n_estimators=300,
                max_depth=None,
                min_samples_leaf=3,
                random_state=seed,
                n_jobs=n_jobs
                )

)

phi_0_cpi_ad_hoc, se_0_cpi_ad_hoc = estimator5.importance(X_subset, y)


print("Feature\tCPI__ad_hoc φ\tStdError")
for j, (phi_j, se_j) in enumerate(zip(phi_0_cpi_ad_hoc, se_0_cpi_ad_hoc)):
    print(f"{j:>3d}\t{phi_j: .4f}\t{se_j: .4f}")
print(f"Sum of CPI__ad_hoc: {D* np.mean(phi_0_cpi_ad_hoc)}")


phi_0_cpi_test_ad_hoc = phi_0_cpi_ad_hoc 

se_0_cpi_test_ad_hoc = se_0_cpi_ad_hoc 

z_score_0_cpi_ad_hoc = phi_0_cpi_test_ad_hoc / se_0_cpi_ad_hoc

p_value_0_cpi_ad_hoc = 1 - norm.cdf(z_score_0_cpi_ad_hoc)
rounded_p_value_0_cpi_ad_hoc = np.round(p_value_0_cpi_ad_hoc, 3)

print(rounded_p_value_0_cpi_ad_hoc)

alpha = 0.05 / 20  

mask = (p_value_0_cpi_ad_hoc <= alpha).astype(int)

print(mask)




d:\ancconda\envs\variable\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
CPI conditional permutation: 100%|██████████| 20/20 [01:12<00:00,  3.64s/it]

Feature	CPI__ad_hoc φ	StdError
  0	 0.0012	 0.0003
  1	 0.0001	 0.0001
  2	 0.0000	 0.0001
  3	 0.0001	 0.0001
  4	 0.0003	 0.0002
  5	 0.0106	 0.0008
  6	 0.0002	 0.0001
  7	 0.0000	 0.0000
  8	 0.0001	 0.0001
  9	 0.0001	 0.0001
 10	 0.0001	 0.0001
 11	 0.0007	 0.0002
 12	 0.0002	 0.0001
 13	 0.0001	 0.0001
 14	 0.0005	 0.0002
 15	 0.0000	 0.0000
 16	 0.0002	 0.0002
 17	 0.0000	 0.0000
 18	 0.0000	 0.0000
 19	 0.0000	 0.0000
Sum of CPI__ad_hoc: 0.016640374802551382
[0.    0.118 0.294 0.016 0.067 0.    0.055 0.5   0.083 0.08  0.109 0.001
 0.111 0.341 0.01  0.5   0.125 0.5   0.5   0.5  ]
[1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier


top_k = 10
D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_0_cpi_ad_hoc, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)


top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by CPI_ad_hoc:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, CPI_ad_hoc={phi_0_cpi_ad_hoc[j]:.6f}, SE={se_0_cpi_ad_hoc[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

seed = seed
n_jobs = n_jobs
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-10 feature indices by CPI_ad_hoc:
  #1: idx=   5, CPI_ad_hoc=0.010597, SE=0.000800
  #2: idx=   0, CPI_ad_hoc=0.001222, SE=0.000264
  #3: idx=  11, CPI_ad_hoc=0.000702, SE=0.000231
  #4: idx=  14, CPI_ad_hoc=0.000519, SE=0.000224
  #5: idx=   4, CPI_ad_hoc=0.000266, SE=0.000177
  #6: idx=  16, CPI_ad_hoc=0.000229, SE=0.000199
  #7: idx=  12, CPI_ad_hoc=0.000170, SE=0.000140
  #8: idx=   6, CPI_ad_hoc=0.000162, SE=0.000101
  #9: idx=   3, CPI_ad_hoc=0.000125, SE=0.000058
  #10: idx=  10, CPI_ad_hoc=0.000103, SE=0.000084
Fold 1 accuracy (selected features): 0.8132
Fold 2 accuracy (selected features): 0.8205
Mean accuracy (2-fold, selected features): 0.8169 ± 0.0052
Selected indices: [5, 0, 11, 14, 4, 16, 12, 6, 3, 10]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 |  0.8169 | ±0.0051 |
| Top-8 | 0.8173 | ±0.0053 |
| Top-6 | 0.8165 | ±0.0048 |
| Top-4 | 0.8118 | ±0.0057 |
| Top-2 | 0.8173| ±0.0053 |

## FDFI

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

top_k = 2

D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_x_cpi, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)

top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by FDFI:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, FDFI={phi_x_cpi[j]:.6f}, SE={se_x_cpi[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

seed = seed
n_jobs = n_jobs
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-2 feature indices by FDFI:
  #1: idx=   1, FDFI=0.069083, SE=0.041050
  #2: idx=   5, FDFI=0.034607, SE=0.001344
Fold 1 accuracy (selected features): 0.8164
Fold 2 accuracy (selected features): 0.8229
Mean accuracy (2-fold, selected features): 0.8197 ± 0.0046
Selected indices: [1, 5]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 | 0.8199 | ±0.0043 |
| Top-8 | 0.8182 | ±0.0034 |
| Top-6 | 0.8185 | ±0.0038 |
| Top-4 | 0.8197 | ±0.0054 |
| Top-2 | 0.8197| ±0.0046 |

## DFI

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

top_k = 6
D = X_full.shape[1]

phi_safe = np.nan_to_num(phi_x_dfi, nan=-np.inf, posinf=-np.inf, neginf=-np.inf)

top_k = min(top_k, D)
selected_feature_idx = np.argsort(phi_safe)[-top_k:][::-1].tolist()

print(f"Top-{top_k} feature indices by DFI:")
for rank, j in enumerate(selected_feature_idx, start=1):
    print(f"  #{rank}: idx={j:4d}, DFI={phi_x_dfi[j]:.6f}, SE={se_x_dfi[j]:.6f}")

X_subset = X_full[:, selected_feature_idx]

seed = seed
n_jobs = n_jobs

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

fold_acc = []
for k, (tr_idx, te_idx) in enumerate(skf.split(X_subset, y), start=1):
    X_tr, X_te = X_subset[tr_idx], X_subset[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]

    clf = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=3,
        random_state=seed,
        n_jobs=n_jobs
    )
    clf.fit(X_tr, y_tr)
    y_pred = clf.predict(X_te)
    acc = accuracy_score(y_te, y_pred)
    fold_acc.append(acc)
    print(f"Fold {k} accuracy (selected features): {acc:.4f}")

print(f"Mean accuracy (2-fold, selected features): {np.mean(fold_acc):.4f} ± {np.std(fold_acc, ddof=1):.4f}")

print("Selected indices:", selected_feature_idx)


Top-6 feature indices by DFI:
  #1: idx=   5, DFI=0.035061, SE=0.002150
  #2: idx=   0, DFI=0.018630, SE=0.001695
  #3: idx=   6, DFI=0.015938, SE=0.001554
  #4: idx=   7, DFI=0.008679, SE=0.001265
  #5: idx=   8, DFI=0.007987, SE=0.001218
  #6: idx=   9, DFI=0.007575, SE=0.001195
Fold 1 accuracy (selected features): 0.8134
Fold 2 accuracy (selected features): 0.8198
Mean accuracy (2-fold, selected features): 0.8166 ± 0.0045
Selected indices: [5, 0, 6, 7, 8, 9]


| Top-k | Accuracy | Std |
|:------|:---------:|:---------:|
| Top-10 | 0.8172 | ±0.0048 |
| Top-8 | 0.8150 | ±0.0051 |
| Top-6 | 0.8166 | ±0.0045 |
| Top-4 | 0.8137 | ±0.0013 |
| Top-2 | 0.8173| ±0.0053 |